Программа повышения квалификации (научно-педагогических) работников НИУ ВШЭ

# Python для исследователей

*Автор: Алла Тамбовцева, НИУ ВШЭ*  

## Web-scraping: скрэйпинг новостного сайта

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с реального сайта [nplus1.ru](https://nplus1.ru/).

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [4]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [5]:
soup = BeautifulSoup(page.text, 'lxml')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [6]:
# soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [7]:
# print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [6]:
for link in soup.find_all('a'):
    print(link.get('href'))

#
/
#
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
/theme/explainatorium
/theme/bookshelf
/theme/nobelprize
/theme/Courses
/theme/coronavirus-history
/
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
#
/theme/explainatorium
/theme/bookshelf
/theme/nobelprize
/theme/Courses
/theme/coronavirus-history
https://nplus1.ru/blog/2020/10/08/the-science-of-staying-well
https://nplus1.ru/blog/2020/10/08/the-science-of-staying-well
https://nplus1.ru/blog/2020/10/07/fight-for-freedom-in-the-don-region
https://nplus1.ru/blog/2020/10/01/october-2020-observations
https://nplus1.ru/blog/2020/09/30/in-search-of-constantinople
https://nplus1.ru/blog/2020/09/28/the-hungry-steppe
https://nplus1.ru/blog/2020/09/24/nobel-prediction
https://nplus1.ru/blog/2020/09/18/formations-of-the-secular
https://nplus1.ru/blog/2020/09/15/the-hidden-history-of-animal-life
https://nplus1.ru/blog/2020/09/11/who-we-are-and-how-we-got-here
https://nplus1.ru/blog/2020/10/0

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [7]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
urls

['/news/2020/10/10/choka',
 '/news/2020/10/10/bone-marrow-gene-silencing',
 '/news/2020/10/10/cephalopod-inspired-robot',
 '/news/2020/10/10/metallization-of-diamond',
 '/news/2020/10/10/occupy-molerat',
 '/news/2020/10/09/ancient-ball',
 '/news/2020/10/09/drone-drop',
 '/news/2020/10/09/paper-picturers',
 '/news/2020/10/09/viruses-bats',
 '/news/2020/10/09/u-2-kubernetes',
 '/news/2020/10/09/3d-metal',
 '/news/2020/10/09/crispr-vs-covid',
 '/news/2020/10/09/aircarrier',
 '/news/2020/10/09/bahamas-birds-extinction',
 '/news/2020/10/09/magneto-thomson-effect',
 '/news/2020/10/09/better-cryoEM',
 '/news/2020/10/09/hapsmobile',
 '/news/2020/10/08/gmo-wheat-argentine',
 '/news/2020/10/08/strange-b-meson-asymmetry',
 '/news/2020/10/08/calcium-fly-trap',
 '/news/2020/10/08/strange-b-meson-asymmetry',
 '/news/2020/10/09/magneto-thomson-effect',
 '/news/2020/10/09/better-cryoEM',
 '/news/2020/10/07/aurora-a-shredding',
 '/news/2020/10/07/nanofountain']

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки (название сайта) отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [8]:
full_urls = []

for u in urls:
    res = 'https://nplus1.ru' + u
    full_urls.append(res) 

full_urls

['https://nplus1.ru/news/2020/10/10/choka',
 'https://nplus1.ru/news/2020/10/10/bone-marrow-gene-silencing',
 'https://nplus1.ru/news/2020/10/10/cephalopod-inspired-robot',
 'https://nplus1.ru/news/2020/10/10/metallization-of-diamond',
 'https://nplus1.ru/news/2020/10/10/occupy-molerat',
 'https://nplus1.ru/news/2020/10/09/ancient-ball',
 'https://nplus1.ru/news/2020/10/09/drone-drop',
 'https://nplus1.ru/news/2020/10/09/paper-picturers',
 'https://nplus1.ru/news/2020/10/09/viruses-bats',
 'https://nplus1.ru/news/2020/10/09/u-2-kubernetes',
 'https://nplus1.ru/news/2020/10/09/3d-metal',
 'https://nplus1.ru/news/2020/10/09/crispr-vs-covid',
 'https://nplus1.ru/news/2020/10/09/aircarrier',
 'https://nplus1.ru/news/2020/10/09/bahamas-birds-extinction',
 'https://nplus1.ru/news/2020/10/09/magneto-thomson-effect',
 'https://nplus1.ru/news/2020/10/09/better-cryoEM',
 'https://nplus1.ru/news/2020/10/09/hapsmobile',
 'https://nplus1.ru/news/2020/10/08/gmo-wheat-argentine',
 'https://nplus1.ru/

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [9]:
url0 = full_urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'lxml')

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [10]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="black" name="apple-mobile-web-app-status-bar-style"/>,
 <meta content="2020-10-10" itemprop="datePublished"/>,
 <meta content="Наталия Миранда" name="mediator_author"/>,
 <meta content="Для этого они создали липид-полимерные наночастицы, несущие siRNA" name="description"/>,
 <meta content="Наталия Миранда" name="author"/>,
 <meta content="" name="copyright"/>,
 <meta content="Биоинженеры научились управлять стволовыми клетками РНК-интерференцией" property="og:title"/>,
 <meta content="https://nplus1.ru/images/2020/10/10/1a6f9078d039cd85e90712c3422a8e16.png" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2020/10/10/bone-marrow-gene-silencing" property="og:url"/>,
 <meta content="Для этого они создали липид-полимерные наночастицы, несущие siRNA" 

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [11]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Наталия Миранда" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [12]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Наталия Миранда" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [13]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Наталия Миранда'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [14]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Наталия Миранда'

Аналогичным образом извлечем дату, заголовок и описание.

In [15]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']

Осталось вытащить рубрики и сложность текста. Если мы посмотрим на исходный код страницы, мы увидим, что нужная нам информация находится в тэгах `<p></p>`:

In [16]:
soup0.find_all('p')

[<p class="table">
 <a data-rubric="materials" href="/rubric/materials">Материалы</a>
 <a data-rubric="medicine" href="/rubric/medicine">Медицина</a>
 </p>, <p class="table">
 <a href="/news/2020/10/10">
 <time content="2020-10-10" data-unix="1602333455" itemprop="datePublished">
 <span>15:37</span>
 <span>10 Окт. 2020</span>
 </time>
 </a>
 </p>, <p class="table">
 <a href="/difficult/3.2">
 <span>Сложность</span>
 <span class="difficult-value">3.2</span>
 </a>
 </p>, <p class="title">Миграция нейтрофилов из ниш стволовых клеток. Слева (верх и низ) — контроль, справа — до обработки наночастицами с siSdf1 (верх) и после (низ)</p>, <p class="credits">Matthias Nahrendorf et al. / Nature Biomedical Engineering, 2020</p>, <p>Американские ученые разработали липид-полимерные наночастицы, несущие малые интерферирующие РНК и способные проникать в клетки стромы костного мозга. Подавляя там экспрессию различных генов, исследователи смогли управлять процессами миграции стволовых и прогениторных к

Выберем из полученного списка первый элемент и найдем в нем все тэги `<a>`:

In [17]:
soup0.find_all('p')[0].find_all('a')

[<a data-rubric="materials" href="/rubric/materials">Материалы</a>,
 <a data-rubric="medicine" href="/rubric/medicine">Медицина</a>]

Получился список из одного элемента. Применим списковые включения – вытащим из каждого элемента (их могло бы быть больше) текст и поместим его в новый список `rubrics`.

In [18]:
rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
rubrics

['Материалы', 'Медицина']

Осталась только сложность. Возьмем соответствующий кусок кода:

In [19]:
soup0.find_all('span', {'class' : 'difficult-value'})

[<span class="difficult-value">3.2</span>]

И выберем оттуда текст.

In [20]:
diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
diff

'3.2'

Теперь перейдем к тексту самой новости. Как можно заметить, текст сохранен в абзацах `<p></p>`, причем безо всяких атрибутов. Сообщим Python, что нас интересуют куски с пустым атрибутом `class`:

In [29]:
text_list = soup0.find_all('p', {'class' : None})

«Выцепим» все тексты (без тэгов) из полученного списка:

In [30]:
text = [t.text for t in text_list]

Склеим все элементы списка `text` через пробел:

In [31]:
final_text = ' '.join(text)
final_text

'Американские ученые разработали липид-полимерные наночастицы, несущие малые интерферирующие РНК и\xa0способные проникать в\xa0клетки стромы костного мозга. Подавляя там экспрессию различных генов, исследователи смогли управлять процессами миграции стволовых и\xa0прогениторных клеток из\xa0костного мозга в\xa0кровоток. Результаты исследования опубликованы в\xa0журнале Nature Biomedical Engineering. В\xa0костном мозге человека находится внушительное количество прогениторных и\xa0гемопоэтических стволовых клеток, а\xa0в\xa0кровоток ежедневно\xa0 высвобождается множество клеток крови. Процессы их\xa0дифференциации, пролиферации и\xa0миграции частично регулируются микроокружением стволовых клеток\xa0— стромой костного мозга, или нишевыми клетками. К\xa0ним относятся эндотелиальные клетки костного мозга, образующие тонкие однослойные кровеносные сосуды, и\xa0способные самостоятельно выделять сигнальные молекулы для кроветворных клеток. Перспектива управления процессами миграции гемопоэтичес

Все здорово, только мешают отступы-переходы на новую строку `\n`. Заменим их на пробелы с помощью метода `.replace`:

In [32]:
final_text = final_text.replace('\n', ' ')

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [33]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']
    rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
    diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
    text_list = soup0.find_all('p', {'class' : None})
    text = [t.text for t in text_list]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', ' ')
    
    return url0, date, author, description, title, final_text, rubrics, diff

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [34]:
from time import sleep

In [35]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in full_urls:
    res = GetNews(link)
    news.append(res)
    sleep(3) # задержка в 3 секунды

Так теперь выглядит первый элемент списка:

In [36]:
news[0]

('https://nplus1.ru/news/2020/10/10/choka',
 '2020-10-10',
 'Николай Воронцов',
 'Раму можно самостоятельно накачать дома насосом или компрессором и не брать с собой в поездку велосипедный насос.',
 'Раму велосипеда превратили в резервуар для сжатого воздуха',
 'Французские дизайнеры предложили использовать велосипедную раму в качестве баллона для сжатого воздуха. По словам авторов проекта на Kickstarter, раму можно самостоятельно накачать дома насосом или компрессором и не брать с собой в поездку велосипедный насос. Велосипедисты нередко берут с собой в поездку велонасос — иногда колесо нужно накачать после ремонта в полевых условиях, а иногда бескамерная покрышка может соскочить с обода. Популярная альтернатива ручным насосам — насосы со сменными картриджами углекислого газа, к которым после каждого использования нужно приобретать новый картридж, поскольку они не подразумевают перезарядки в домашних условиях. Существуют и более экзотические решения — например, портативный многоразовы

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame(news)

In [39]:
df.head(2)

,0,1,2,3,4,5,6,7
0,https://nplus1.ru/news/2020/10/10/choka,2020-10-10,Николай Воронцов,Раму можно самостоятельно накачать дома насосо...,Раму велосипеда превратили в резервуар для сжа...,Французские дизайнеры предложили использовать ...,[Транспорт],1.1
1,https://nplus1.ru/news/2020/10/10/bone-marrow-...,2020-10-10,Наталия Миранда,Для этого они создали липид-полимерные наночас...,Биоинженеры научились управлять стволовыми кле...,Американские ученые разработали липид-полимерн...,"[Материалы, Медицина]",3.2


Переименуем столбцы в базе.

In [40]:
df.columns = ['link', 'date', 'author', 'desc', 'title', 'text', 'rubric', 'diffc']

In [41]:
df.head(2)

,link,date,author,desc,title,text,rubric,diffc
0,https://nplus1.ru/news/2020/10/10/choka,2020-10-10,Николай Воронцов,Раму можно самостоятельно накачать дома насосо...,Раму велосипеда превратили в резервуар для сжа...,Французские дизайнеры предложили использовать ...,[Транспорт],1.1
1,https://nplus1.ru/news/2020/10/10/bone-marrow-...,2020-10-10,Наталия Миранда,Для этого они создали липид-полимерные наночас...,Биоинженеры научились управлять стволовыми кле...,Американские ученые разработали липид-полимерн...,"[Материалы, Медицина]",3.2


Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [42]:
df.to_excel('nplus-news.xlsx')